In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns
from scipy import stats
import tensorboard as tb
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator
import os
from tqdm import tqdm

In [ ]:
sns.set(font_scale=1.5)

In [ ]:
# https://github.com/theRealSuperMario/supermariopy/blob/master/scripts/tflogs2pandas.py
def tflog2pandas(path: str) -> pd.DataFrame:
    """convert single tensorflow log file to pandas DataFrame
    Parameters
    ----------
    path : str
        path to tensorflow log file
    Returns
    -------
    pd.DataFrame
        converted dataframe
    """
    DEFAULT_SIZE_GUIDANCE = {
        "compressedHistograms": 1,
        "images": 1,
        "scalars": 0,  # 0 means load all
        "histograms": 1,
    }
    runlog_data = pd.DataFrame({"metric": [], "value": [], "step": []})
    try:
        event_acc = EventAccumulator(path, DEFAULT_SIZE_GUIDANCE)
        event_acc.Reload()
        tags = event_acc.Tags()["scalars"]
        for tag in tags:
            event_list = event_acc.Scalars(tag)
            values = list(map(lambda x: x.value, event_list))
            step = list(map(lambda x: x.step, event_list))
            r = {"metric": [tag] * len(step), "value": values, "step": step}
            r = pd.DataFrame(r)
            runlog_data = pd.concat([runlog_data, r])
    # Dirty catch of DataLossError
    except Exception:
        print("Event file possibly corrupt: {}".format(path))
        traceback.print_exc()
    return runlog_data


def many_logs2pandas(event_paths):
    all_logs = pd.DataFrame()
    for path in event_paths:
        log = tflog2pandas(path)
        if log is not None:
            if all_logs.shape[0] == 0:
                all_logs = log
            else:
                all_logs = all_logs.append(log, ignore_index=True)
    return all_logs

In [ ]:
def load_models(dir):
    all_logs = pd.DataFrame()
    models = next(os.walk(dir))[1]
    for model in tqdm(models):
        model_path = os.path.join(dir, model)
        runs = next(os.walk(model_path))[1]
        for run in runs:
            path = os.path.join(model_path, run)
            log = tflog2pandas(path)
            log['run'] = run
            log['model'] = model
            # string_splits = run.split('_')
            # log['tag'] = string_splits[0]
            # log['model'] = string_splits[1]
            # hp = ', '.join(string_splits[2:-3])
            # if string_splits[1] == 'GecoVAE':
            #     hp = hp.replace(", 255,", ",")
            #     hp = hp.replace("lambda, init", "lambda_init")
            # log['hyperparams'] = hp
            # log['seed'] = string_splits[-3][4:]
            # log['date'] = string_splits[-2] + "_" + string_splits[-1]

            if log is not None:
                if all_logs.shape[0] == 0:
                    all_logs = log
                else:
                    all_logs = all_logs.append(log, ignore_index=True)


    # create dataframe with final values
    # dff = all_logs[all_logs.step == all_logs.step.max()]
    # dff = dff.pivot_table(index=["run", "tag", "model", "hyperparams", "seed", "date"], columns=["metric"],values="value")
    # dff = pd.DataFrame(dff.to_records())
    return all_logs #, dff

In [ ]:
# dir = './carla/agents/data/'
# df = load_models(dir)

In [ ]:
# df.to_pickle("invers_exp.pkl")

In [ ]:
df = pd.read_pickle("all_ncextra.pkl")

In [ ]:
a = df
a = a[a.metric == "eval_stats/AvgEpRet"]
a = a[a.step == a.step.max()]


In [ ]:
a.sort_values("value")

In [ ]:
plt.figure(figsize=(16, 18))
sns.boxplot(data=a, y=a.model, x=a.value).set_title("Cumulative Reward")


In [ ]:
import re
a['model_arch'] = a.model.apply(lambda x: 'nc' if x == 'nc' else re.findall(r'carlac_frozenvae_(\w+)_',x)[0])
a['model_hp'] = a.model.apply(lambda x: '' if x == 'nc' else re.findall(r'carlac_frozenvae_[^_]+(.*)_seed',x)[0])

In [ ]:
df_mean = a.groupby(["model_arch","model_hp"]).mean()

In [ ]:
df_mean

In [ ]:
green = sns.light_palette("green", as_cmap=True)
df_mean.style\
    .background_gradient(cmap=green, subset=['value'])

In [ ]:
df_std = a.groupby(["model_arch","model_hp"]).agg({'value': ['mean', 'std', 'max', 'count']})

In [ ]:
df_std

In [ ]:
plt.figure(figsize=(16, 9))
sns.boxplot(data=a, y=a["model+hp"], x=a.value).set_title("Cumulative Reward")


In [ ]:
################################################################################################
################################################################################################
################################################################################################
################################################################################################

In [ ]:
a = df
a = a[a.metric == "eval_stats/AvgEpRet"]
a = a[a.step == a.step.max()]

import re
def desc_columns(a):
    a['arch'] = a.model.apply(lambda x: 'nc' if x == 'nc' else '[Eghbal-zadeh et al., 2021]' if x == "carla45fully48px_vpg_3k_concatNoEncode_gco_vaefrozen_pmlr_all" else re.findall(r'carlac_frozenvae_(\w+)_',x)[0])
    a['hp'] = a.model.apply(lambda x: '' if x == 'nc' or x == "carla45fully48px_vpg_3k_concatNoEncode_gco_vaefrozen_pmlr_all" else re.findall(r'carlac_frozenvae_[^_]+_(.*)_seed',x)[0])

    def simplifyHp(x):
        matches = re.findall(r'gamma100\.0_capacity([0-9.]+)_iterations10000',x)
        if matches.__len__() > 0:
            x = "capacity_"+matches[0]

        matches = re.findall(r'tolerance([0-9.]+)_255_lambda_init([0-9.]+)_step500',x)
        if matches.__len__() > 0:
            x = "tolerance"+matches[0][0] + " lambda-init_"+matches[0][1]
        
        return x

    a['hp'] = a['hp'].apply(simplifyHp)
    a['name'] = a.arch +" "+ a.hp

    return a

a = desc_columns(a)

In [ ]:
a

In [ ]:
a.groupby(["name"]).agg({'value': ['mean', 'std', 'max', "min", 'count']})

In [ ]:
print(a.groupby(["name"]).agg({'value': ['mean', 'std', 'max', "min", 'count']}).to_latex())

In [ ]:
plt.figure(figsize=(16, 9))
sns.boxplot(data=a, y=a["name"], x=a.value).set_title("Cumulative Reward")


In [ ]:
a.name.unique()

In [ ]:
plt.figure(figsize=(9, 4))
sns.boxplot(data=a[a.arch == "BetaVAE"], y="name", x="value", order=["BetaVAE beta1.0", "BetaVAE beta5.0","BetaVAE beta10.0","BetaVAE beta20.0"]).set(xlabel="Cumulative Reward", ylabel=None)


In [ ]:
plt.figure(figsize=(9, 3))
sns.boxplot(data=a[a.arch == "AnnealedVAE"], y="name", x="value", ).set(xlabel="Cumulative Reward", ylabel=None)


In [ ]:
plt.figure(figsize=(9, 4))
sns.boxplot(data=a[a.arch == "GecoVAE"], y="name", x="value").set(xlabel="Cumulative Reward", ylabel=None)


In [ ]:
clist = ["nc ", "[Eghbal-zadeh et al., 2021] ", "BetaVAE beta20.0", "AnnealedVAE capacity_15.0", "GecoVAE tolerance5.0 lambda-init_0.1"]
c = a[a.name.isin(clist)]
plt.figure(figsize=(9, 6))
order = clist
sns.boxplot(data=c, y="name", x="value", order=order).set(xlabel="Cumulative Reward", ylabel=None)


In [ ]:
df = desc_columns(df)

In [ ]:
tlist = ["nc ", "[Eghbal-zadeh et al., 2021] ", "GecoVAE tolerance5.0 lambda-init_0.1"]
t = df[df.name.isin(tlist)]

In [ ]:
t

In [ ]:
sns.lineplot(data=t[t.metric == "eval_stats/AvgEpRet"], x="step", y="value", hue="name").set_title("reward by Config")


In [ ]:
sns.lineplot(data=t[t.metric == "eval_stats/GOD_0.7"], x="step", y="value", hue="name").set_title("GOD0.7 by Config")


In [ ]:
df

In [ ]:
#df.to_pickle("rl.pkl")

In [ ]:
clist = ["nc ", "[Eghbal-zadeh et al., 2021] ", "BetaVAE beta20.0", "AnnealedVAE capacity_15.0", "GecoVAE tolerance5.0 lambda-init_0.1"]
a = df
a = a[a.metric == "eval_stats/AvgScore"]
a = a[a.step == a.step.max()]
cgod = a[a.name.isin(clist)]
plt.figure(figsize=(9, 6))
order = clist
sns.boxplot(data=cgod, y="name", x="value", order=order).set(xlabel="Avg GOD Score", ylabel=None)


In [ ]:
print(cgod.groupby('name').mean().to_latex())

In [ ]:
cgod[cgod['arch'] == "nc"]

In [ ]:
cgod[cgod['arch'] == "[Eghbal-zadeh et al., 2021]"][['run', 'value']].values

In [ ]:
cgod[cgod['arch'] == "nc"]['value'].mean(), cgod[cgod['arch'] == "[Eghbal-zadeh et al., 2021]"]['value'].mean()

In [ ]:
clist = ["nc ", "[Eghbal-zadeh et al., 2021] ", "BetaVAE beta20.0", "AnnealedVAE capacity_15.0", "GecoVAE tolerance5.0 lambda-init_0.05", "GecoVAE tolerance5.0 lambda-init_0.1"]
a = df
a = a[a.metric == "eval_stats/GOD_0.7"]
a = a[a.step == a.step.max()]
cg = a[a.name.isin(clist)]
plt.figure(figsize=(9, 6))
order = clist
sns.boxplot(data=cg, y="name", x="value", order=order).set(xlabel="GOD_0.7", ylabel=None)


In [ ]:
t = df
t = t[t.metric == "eval_stats/AvgScore"]
t = t[t.step == 0]
t[t['name'] == "AnnealedVAE capacity_15.0"]["run"].values

In [ ]:
from scipy.stats import mannwhitneyu
from scipy.stats import ttest_ind

In [ ]:
s = df
s = s[s.metric == "eval_stats/AvgScore"]
s = s[s.step == 59]

In [ ]:
blist = ["nc ", "[Eghbal-zadeh et al., 2021] "]
clist = ["[Eghbal-zadeh et al., 2021] ", "BetaVAE beta20.0", "AnnealedVAE capacity_15.0", "GecoVAE tolerance10.0 lambda-init_0.1",  "GecoVAE tolerance5.0 lambda-init_0.1"]
s[s['name'] == blist[-1]].value
for model in clist:
 print(model, "...", mannwhitneyu(s[s['name'] == blist[-1]].value, s[s['name'] ==model].value))

In [ ]:
blist = ["nc ", "[Eghbal-zadeh et al., 2021] "]
clist = ["[Eghbal-zadeh et al., 2021] ", "BetaVAE beta20.0", "AnnealedVAE capacity_15.0", "GecoVAE tolerance5.0 lambda-init_0.1"]
s[s['name'] == blist[-1]].value
for model in clist:
 print(model, "...", ttest_ind(s[s['name'] == blist[-1]].value, s[s['name'] ==model].value))

In [ ]:
s = df
s = s[s.metric == "eval_stats/AvgScore"]
s = s[s.step == 29]